In [1]:
from contrans import contrans

In [2]:
import numpy as np
import pandas as pd
import json

In [3]:
ct = contrans()

In [4]:
useragent = ct.get_useragent()
useragent

'python-requests/2.32.3'

In [5]:
headers = ct.make_headers(useragent)

In [6]:
goodbioguide = ct.get_bioguide(name="good", state="Virginia", district=5)
goodbioguide

,bioguideId,district,name,partyName,state,updateDate,url,depiction.attribution,depiction.imageUrl,terms.item
0,G000595,5.0,"good, bob",Republican,Virginia,2024-06-08T18:40:22Z,https://api.congress.gov/v3/member/G000595?for...,Image courtesy of the Member,https://www.congress.gov/img/member/g000595_20...,"[{'chamber': 'House of Representatives', 'star..."
1,G000280,5.0,"goode, virgil h., jr.",Republican,Virginia,2022-12-16T21:56:51Z,https://api.congress.gov/v3/member/G000280?for...,"Image, Congressional Pictorial Directory, 109th.",https://www.congress.gov/img/member/g000280_20...,"[{'chamber': 'House of Representatives', 'endY..."


In [7]:
goodid = goodbioguide["bioguideId"][0]

In [8]:
goodid

'G000595'

### Sponsored Legislation

In [12]:
import requests 

root = "https://api.congress.gov/v3"
endpoint = f"/member/{goodid}/sponsored-legislation"
header = ct.make_headers()

params = {"api_key": ct.congresskey,
          "limit": 250}

r = requests.get(root + endpoint, 
                    params=params, 
                    headers=headers)


In [15]:
json.loads(r.text)["sponsoredLegislation"][2]

{'congress': 118,
 'introducedDate': '2024-05-23',
 'latestAction': {'actionDate': '2024-07-05',
  'text': 'Placed on the Union Calendar, Calendar No. 474.'},
 'number': '8534',
 'policyArea': {'name': None},
 'title': 'Protecting Student Athletes’ Economic Freedom Act of 2024',
 'type': 'HR',
 'url': 'https://api.congress.gov/v3/bill/118/hr/8534?format=json'}

In [14]:
house

,congress,chamber,rollnumber,icpsr,cast_code,prob
0,118,House,1,14854,6,100.0
1,118,House,1,14863,6,98.2
2,118,House,1,14873,1,100.0
3,118,House,1,15029,1,100.0
4,118,House,1,15433,1,100.0
...,...,...,...,...,...,...
486659,118,House,1118,31101,6,100.0
486660,118,House,1118,31102,9,99.6
486661,118,House,1118,39301,6,100.0
486662,118,House,1118,39307,6,100.0


In [16]:
house = house[["rollnumber", "icpsr", "cast_code"]]

In [17]:
house_mat = pd.merge(house, house, on="rollnumber")

In [19]:
len(house_mat)

211856536

In [20]:
house_mat["agree"] = house_mat["cast_code_x"] == house_mat["cast_code_y"]

In [22]:
house_mat = house_mat.groupby(["icpsr_x", "icpsr_y"]).agg({"agree": "mean"})

In [23]:
house_mat = house_mat.reset_index()

In [24]:
house_mat

,icpsr_x,icpsr_y,agree
0,14854,14854,1.000000
1,14854,14863,0.830948
2,14854,14873,0.386404
3,14854,15029,0.412343
4,14854,15433,0.366726
...,...,...,...
200607,91980,31102,0.731664
200608,91980,31103,0.190805
200609,91980,39301,0.236136
200610,91980,39307,0.239714


In [25]:
ideo = ct.get_ideology()

In [26]:
members = ideo[["bioname", "icpsr", "party_code"]]

In [27]:
members

,bioname,icpsr,party_code
0,"ROGERS, Mike Dennis",20301,200
1,"SEWELL, Terri",21102,100
2,"PALMER, Gary James",21500,200
3,"CARL, Jerry L.",22108,200
4,"MOORE, Barry",22140,200
...,...,...,...
443,"STEIL, Bryan",21970,200
444,"TIFFANY, Thomas P.",21989,200
445,"FITZGERALD, Scott",22115,200
446,"VAN ORDEN, Derrick",22370,200
